In [5]:
#import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error



### deribit API Request

In [4]:
""" Download CC-Option Data from Deribit via public API """

"""
Matteo Bottacini -- matteo.bottacini@usi.ch
"""

# import modules
import json
import requests
import pandas as pd
from tqdm import tqdm
import sqlite3
import datetime


# functions
def get_option_name_and_settlement(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: 2 lists:
                        1.  list of traded options for the selected coin;
                        2.  list of settlement period for the selected coin.
    """

    # requests public API
    r = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=" + coin + "&kind=option")
    result = json.loads(r.text)

    # get option name
    name = pd.json_normalize(result['result'])['instrument_name']
    name = list(name)

    # get option settlement period
    settlement_period = pd.json_normalize(result['result'])['settlement_period']
    settlement_period = list(settlement_period)

    return name, settlement_period


def get_option_data(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: pandas data frame with all option data for a given coin
    """

    # get option name and settlement
    coin_name = get_option_name_and_settlement(coin)[0]
    settlement_period = get_option_name_and_settlement(coin)[1]

    # initialize data frame
    coin_df = []

    # initialize progress bar
    pbar = tqdm(total=len(coin_name))

    # loop to download data for each Option Name
    for i in range(len(coin_name)):
        # download option data -- requests and convert json to pandas
        r = requests.get('https://test.deribit.com/api/v2/public/get_order_book?instrument_name=' + coin_name[i])
        result = json.loads(r.text)
        df = pd.json_normalize(result['result'])

        # add settlement period
        df['settlement_period'] = settlement_period[i]

        # append data to data frame
        coin_df.append(df)

        # update progress bar
        pbar.update(1)

    # finalize data frame
    coin_df = pd.concat(coin_df)

    # remove useless columns from coin_df
    columns = ['state', 'estimated_delivery_price']
    coin_df.drop(columns, inplace=True, axis=1)

    # close the progress bar
    pbar.close()

    return coin_df




In [5]:
# print data and time for log
print('Date and time: ' +  datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S") + ' , format: dd/mm/yyyy hh:mm:ss')

# download data -- BTC and ETH Options
btc_data = get_option_data('BTC')
eth_data = get_option_data('ETH')

# The data is now stored in Pandas DataFrames 'btc_data' and 'eth_data'
print('BTC data collected')
print(btc_data.head()) # Display first few rows of BTC data
print('ETH data collected')
print(eth_data.head()) # Display first few rows of ETH data

Date and time: 27/01/2024 20:48:31 , format: dd/mm/yyyy hh:mm:ss


100%|██████████| 684/684 [04:02<00:00,  2.89it/s]c:\Users\Ethan Vaz Falcao\anaconda3\lib\site-packages\pandas\core\internals\concat.py:382: RuntimeWarning: coroutine 'call_api' was never awaited
  unit = JoinUnit(blk, shape)
100%|██████████| 606/606 [03:39<00:00,  2.76it/s]

BTC data collected
   best_bid_amount  best_ask_amount  bid_iv  ask_iv underlying_index  \
0              0.0             10.0     0.0  302.28  SYN.BTC-28JAN24   
0              0.0             10.0     0.0  213.97  SYN.BTC-28JAN24   
0              0.0             10.0     0.0  270.94  SYN.BTC-28JAN24   
0              0.0             10.0     0.0  195.70  SYN.BTC-28JAN24   
0              0.0             10.0     0.0  263.13  SYN.BTC-28JAN24   

   underlying_price  mark_iv  interest_rate  best_bid_price  best_ask_price  \
0        42117.9379   104.62            0.0             0.0          0.1345   
0        42119.0537   104.62            0.0             0.0          0.0001   
0        42119.0537   101.32            0.0             0.0          0.1225   
0        42119.0537   101.32            0.0             0.0          0.0001   
0        42119.5533   100.23            0.0             0.0          0.1110   

   ...  greeks.theta  greeks.vega  greeks.gamma  greeks.delta  \
0  ...  

In [11]:
btc_data.to_csv('data/btc_data.csv', index=False)
eth_data.to_csv('data/eth_data.csv', index=False)

In [6]:
btc_data.head()

,best_bid_amount,best_ask_amount,bid_iv,ask_iv,underlying_index,underlying_price,mark_iv,interest_rate,best_bid_price,best_ask_price,...,greeks.theta,greeks.vega,greeks.gamma,greeks.delta,stats.volume_usd,stats.volume,stats.price_change,stats.low,stats.high,settlement_period
0,0.0,10.0,0.0,302.28,SYN.BTC-28JAN24,42117.9379,104.62,0.0,0.0,0.1345,...,-0.00038,0.00001,0.0,1.00000,0.0,0.0,None,None,None,day
0,0.0,10.0,0.0,213.97,SYN.BTC-28JAN24,42119.0537,104.62,0.0,0.0,0.0001,...,-0.00003,0.00001,0.0,0.00000,0.0,0.0,None,None,None,day
0,0.0,10.0,0.0,270.94,SYN.BTC-28JAN24,42119.0537,101.32,0.0,0.0,0.1225,...,-0.00197,0.00004,0.0,1.00000,0.0,0.0,None,None,None,day
0,0.0,10.0,0.0,195.70,SYN.BTC-28JAN24,42119.0537,101.32,0.0,0.0,0.0001,...,-0.00015,0.00004,0.0,0.00000,0.0,0.0,None,None,None,day
0,0.0,10.0,0.0,263.13,SYN.BTC-28JAN24,42119.5533,100.23,0.0,0.0,0.1110,...,-0.01554,0.00031,0.0,0.99999,0.0,0.0,None,None,None,day


In [9]:
btc_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 684 entries, 0 to 0
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   best_bid_amount     684 non-null    float64
 1   best_ask_amount     684 non-null    float64
 2   bid_iv              684 non-null    float64
 3   ask_iv              684 non-null    float64
 4   underlying_index    684 non-null    object 
 5   underlying_price    684 non-null    float64
 6   mark_iv             684 non-null    float64
 7   interest_rate       684 non-null    float64
 8   best_bid_price      684 non-null    float64
 9   best_ask_price      684 non-null    float64
 10  mark_price          684 non-null    float64
 11  open_interest       684 non-null    float64
 12  max_price           684 non-null    float64
 13  min_price           684 non-null    float64
 14  settlement_price    642 non-null    float64
 15  last_price          427 non-null    object 
 16  asks      

In [7]:
eth_data.head()

,best_bid_amount,best_ask_amount,bid_iv,ask_iv,underlying_index,underlying_price,mark_iv,interest_rate,best_bid_price,best_ask_price,...,greeks.theta,greeks.vega,greeks.gamma,greeks.delta,stats.volume_usd,stats.volume,stats.price_change,stats.low,stats.high,settlement_period
0,0.0,0.0,0.0,0.0,SYN.ETH-28JAN24,2276.8832,222.73,0.0,0.0,0.0,...,-2.21171,0.01986,0.00025,0.98721,0.00,0.0,None,None,None,day
0,0.0,0.0,0.0,0.0,SYN.ETH-28JAN24,2276.9332,222.73,0.0,0.0,0.0,...,-0.60758,0.01984,0.00025,-0.01278,0.00,0.0,None,None,None,day
0,0.0,0.0,0.0,0.0,SYN.ETH-28JAN24,2276.9332,222.73,0.0,0.0,0.0,...,-5.16421,0.04637,0.00058,0.96511,0.00,0.0,None,None,None,day
0,0.0,0.0,0.0,0.0,SYN.ETH-28JAN24,2276.9332,222.73,0.0,0.0,0.0,...,-1.88950,0.04637,0.00058,-0.03489,45.27,10.0,0.0,0.002,0.002,day
0,0.0,0.0,0.0,0.0,SYN.ETH-28JAN24,2276.9332,222.73,0.0,0.0,0.0,...,-9.97799,0.08960,0.00111,0.91982,0.00,0.0,None,None,None,day


In [10]:
eth_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 606 entries, 0 to 0
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   best_bid_amount     606 non-null    float64
 1   best_ask_amount     606 non-null    float64
 2   bid_iv              606 non-null    float64
 3   ask_iv              606 non-null    float64
 4   underlying_index    606 non-null    object 
 5   underlying_price    606 non-null    float64
 6   mark_iv             606 non-null    float64
 7   interest_rate       606 non-null    float64
 8   best_bid_price      606 non-null    float64
 9   best_ask_price      606 non-null    float64
 10  mark_price          606 non-null    float64
 11  open_interest       606 non-null    float64
 12  max_price           606 non-null    float64
 13  min_price           606 non-null    float64
 14  settlement_price    582 non-null    float64
 15  last_price          373 non-null    object 
 16  asks      